In [99]:
import numpy as np
import math
import datetime

In [3]:
@np.vectorize
def sigmoid(x):
    return 1/(1 + math.exp(-x))
@np.vectorize
def sigmoidGradient(x):
    return sigmoid(x)*(1 - sigmoid(x))

In [4]:
#test if numerical gradient is somewhere near computed one
def numerical_grad():
    def J_func(theta):
        return compute_cost(X, y, theta)[0]
    numgrad = np.zeros_like(weights)
    perturb = np.zeros_like(weights)
    e = 0.0001
    for p in range(perturb.size):
        perturb[p] = e
        loss1 = J_func(weights - perturb)
        loss2 = J_func(weights + perturb)
        numgrad[p] = (loss2 - loss1) / (2 * e)
        perturb[p] = 0
    return numgrad

In [111]:
class NeuralNetwork3:
    def __init__(self, input_layer_size, hidden_layer_size, output_layer_size):
        self.input_layer_size = input_layer_size
        self.hidden_layer_size = hidden_layer_size
        self.output_layer_size = output_layer_size
        self.theta1_size = (hidden_layer_size, input_layer_size + 1)
        self.theta2_size = (output_layer_size, hidden_layer_size + 1)
    
    def train(self, X, y, iterations, learning_rate):
        self._init_weights()
        self.errors, self.weights = self._gradient_descent(X, y, self.weights, iterations=iterations, learning_rate=learning_rate)
    
    def _init_weights(self):
        Theta1 = np.random.random_sample(self.theta1_size)
        Theta2 = np.random.random_sample(self.theta2_size)
        self.weights = np.r_[Theta1.reshape(Theta1.size, 1), Theta2.reshape(Theta2.size, 1)].squeeze()
    
    def predict(self, X):
        m = X.shape[0]
        #unfold weights
        weights_counts = (self.theta1_size[0] * self.theta1_size[1], self.theta2_size[0] * self.theta2_size[1])
        Theta1 = self.weights[:weights_counts[0]].reshape(self.theta1_size)
        Theta2 = self.weights[weights_counts[0]:].reshape(self.theta2_size)
        #forward propagation
        a1 = np.hstack((np.ones((m, 1)), X))
        z2 = a1.dot(Theta1.T)
        a2 = np.hstack((np.ones((m, 1)), sigmoid(z2)))
        z3 = a2.dot(Theta2.T)
        h = sigmoid(z3)
        return h
    
    def _compute_cost(self, X, y, weights):
        #number of samples
        m = X.shape[0]
        #unfold weights
        weights_counts = (self.theta1_size[0] * self.theta1_size[1], self.theta2_size[0] * self.theta2_size[1])
        Theta1 = weights[:weights_counts[0]].reshape(self.theta1_size)
        Theta2 = weights[weights_counts[0]:].reshape(self.theta2_size)
        #forward propagation
        a1 = np.hstack((np.ones((m, 1)), X))
        z2 = a1.dot(Theta1.T)
        a2 = np.hstack((np.ones((m, 1)), sigmoid(z2)))
        z3 = a2.dot(Theta2.T)
        h = sigmoid(z3)
        #accumulator for error
        s = 0
        #accumulators for gradient
        DELTA1 = np.zeros_like(Theta1)
        DELTA2 = np.zeros_like(Theta2)
        #todo vectorize (tbh, it's going to be extremely hard)
        for i in range(m):
            #hella long T.T
            s = s - y[i].reshape(1, y[i].size).dot(np.vectorize(math.log)(h[i].reshape(1, h[i].size).T)) - (1 - y[i]).reshape(1, y[i].size).dot(np.vectorize(math.log)((1 - h[i]).reshape(1, h[i].size).T))
            #backpropagation
            d3 = (h[i] - y[i]).T.reshape(self.output_layer_size, 1)
            part1 = Theta2.T.dot(d3)
            part2 = sigmoidGradient(np.c_[np.zeros((1, 1)), z2[i].reshape(1, z2[i].size)]).T
            d2 = (part1*part2)[1:,]
            DELTA1 += d2.dot(a1[i].reshape(1, a1[i].size))
            DELTA2 += d3.dot(a2[i].reshape(1, a2[i].size))
        #values to return
        J = float(s / m)
        Theta1_grad = DELTA1 / m
        Theta2_grad = DELTA2 / m
        #todo add regularization
        #fold weights
        weights_grad = np.r_[Theta1_grad.reshape(Theta1_grad.size, 1), Theta2_grad.reshape(Theta2_grad.size, 1)].squeeze()
        return (J, weights_grad)
    
    def _gradient_descent(self, X, y, weights, iterations=5000, learning_rate=1):
        errors = []
        for i in range(iterations):
            start = datetime.datetime.now()
            J, grad = self._compute_cost(X, y, weights)
            errors.append(J)
            weights = weights - learning_rate * grad
            end = datetime.datetime.now()
            print(end - start, ' per iteration', end='\r')
        return (errors, weights)

In [80]:
import os
from mnist import MNIST
mndata = MNIST(os.getcwd() + '/python-mnist/data')
images, labels = mndata.load_training()

In [ ]:
#returns (inputs, labels)
def load_data():
    m_images = np.array(images).reshape(60000, 28*28) / 255
    m_labels = np.zeros((60000, 10))
    for i in range(len(labels)):
        m_labels[i,labels[i]] = 1
    ids = np.random.randint(0, high=60000, size=5000)
    m_X = np.zeros((5000, 28*28))
    m_y = np.zeros((5000, 10))
    for i in range(5000):
        m_X[i] = m_images[ids[i]]
        m_y[i] = m_labels[ids[i]]
    return (m_X, m_y)

In [112]:
net = NeuralNetwork3(28*28, 25, 10)

In [106]:
X, y = load_data()

In [118]:
net.train(X, y, 5, 1)

In [119]:
net.predict(X[13].reshape(1, 28*28))

array([[ 0.04556521,  0.02210385,  0.08861638,  0.1747068 ,  0.10988941,
         0.1129123 ,  0.14615729,  0.18670378,  0.15773331,  0.11996575]])

In [117]:
y[13]

array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.])